In [73]:
#! pip install beautifulsoup4, requests
# !pip install selenium
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import numpy as np
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support import ui
import os
import sys
os.path.dirname(sys.executable)
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os

In [95]:
url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Milwaukee%2C%20WI&start=230"
html = requests.get(url)
html.content[0:500]

b'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel='

In [96]:
# Making the soup
from bs4 import BeautifulSoup
soup = BeautifulSoup(html.content, 'lxml')

In [150]:
# Getting everything also for multiple pages (here, for page 2, 3, 4, 5, and 6)
url_yelp = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Milwaukee%2C%20WI&start="


# we initialize the links
full_links = []
full_restaurants = []

for i in range(0,240,10):
    
    html = requests.get(url_yelp+str(i))
    
    soup = BeautifulSoup(html.content, 'lxml')
    link = soup.select(".css-166la90")
    
    # Getting the restaurant names and the restaurant links in solation
    name_restaurant = []
    link_restaurant = []

   
    for l in link:
        name_restaurant.append(l.string)
        link_restaurant.append(l.get('href'))
     
    #Here we only ectract the first 10 entries
    full_restaurants.extend(name_restaurant[0:10])
    full_links.extend(link_restaurant[0:10])
    
print(full_restaurants)

['Swingin’ Door Exchange', 'Story Hill BKC', 'Mad Rooster Cafe', 'Maxie’s', 'View MKE', 'Tusk', 'Triciclo Peru', 'St Paul Fish Company', 'Lazo’s Taco Shack', 'Five O’Clock Steakhouse', 'Blue Bat Kitchen and Tequilaria', 'Odd Duck', 'Public Table', 'Third Coast Provisions', 'Fool’s Errand', 'The Capital Grille', 'SweetDiner', 'Blue’s Egg - Milwaukee', 'Amilinda', 'Toast', 'Harbor House', 'La Merenda', 'Sobelman’s Pub & Grill', 'Uncle Wolfie’s Breakfast Tavern', 'Smoke Shack', 'Tangled Noodles', 'Momo Mee', 'Camino', 'The Tandem', 'MOVIDA - Hotel Madrid', 'Char’d', 'DanDan', 'The Noble', 'Le Reve Patisserie & Café', 'Tavo’s Signature Cuisine', 'Ca’Lucchenzo', 'Lowcountry Milwaukee', 'Morel', 'The Diplomat', 'The Explorium Brewpub Third Ward', 'Oscar’s Pub & Grill', 'Bavette', 'San Giorgio Pizzeria Napoletana', 'Buckley’s Restaurant & Bar', 'Stella Van Buren', 'Oak Barrel Public House', 'The Fitz', 'The Chef’s Table', 'Tied House Milwaukee', 'Café Benelux', 'Birch + Butcher', 'The Origina

In [151]:
len(full_restaurants)

240

In [152]:
#then we zip it together and store it in a dataframe
entry_list = list(zip(full_restaurants, full_links))
            
df_mainlinks = pd.DataFrame(entry_list,columns=['restaurant', 'link'])

In [154]:
df_mainlinks.to_csv('mainlinks.csv')

In [3]:
df_new = pd.read_csv('mainlinks.csv')

In [5]:
df_new

,Unnamed: 0,restaurant,link
0,0,Swingin’ Door Exchange,/biz/swingin-door-exchange-milwaukee?osq=Resta...
1,1,Story Hill BKC,/biz/story-hill-bkc-milwaukee?osq=Restaurants
2,2,Mad Rooster Cafe,/biz/mad-rooster-cafe-west-milwaukee?osq=Resta...
3,3,Maxie’s,/biz/maxies-milwaukee?osq=Restaurants
4,4,View MKE,/biz/view-mke-milwaukee?osq=Restaurants
...,...,...,...
235,235,Valley Inn,/biz/valley-inn-milwaukee?osq=Restaurants
236,236,Rice N Roll Bistro,/biz/rice-n-roll-bistro-milwaukee?osq=Restaurants
237,237,Crave Cafe,/biz/crave-cafe-shorewood?osq=Restaurants
238,238,Deli 1614,/biz/deli-1614-milwaukee?osq=Restaurants


In [6]:
#let's us regex to clean the df
import re

In [7]:
#we convert everyting to strings, and then exclude those rows where the column Links contains a https
df_new['restaurant'] = df_new.restaurant.astype('str') 
df_new['link'] = df_new.link.astype('str') 

In [8]:
'''then we complete the links colum for the full URL'''

df_new['Complete_links']= df_new['link'].apply(lambda x:"https://www.yelp.com"+str(x))

In [9]:
df_new.pop('link')

0      /biz/swingin-door-exchange-milwaukee?osq=Resta...
1          /biz/story-hill-bkc-milwaukee?osq=Restaurants
2      /biz/mad-rooster-cafe-west-milwaukee?osq=Resta...
3                  /biz/maxies-milwaukee?osq=Restaurants
4                /biz/view-mke-milwaukee?osq=Restaurants
                             ...                        
235            /biz/valley-inn-milwaukee?osq=Restaurants
236    /biz/rice-n-roll-bistro-milwaukee?osq=Restaurants
237            /biz/crave-cafe-shorewood?osq=Restaurants
238             /biz/deli-1614-milwaukee?osq=Restaurants
239    /biz/riverfront-pizzeria-bar-and-grill-milwauk...
Name: link, Length: 240, dtype: object

In [10]:
len(df_new) 

240

In [12]:
# we see that there are two duplicate restaurants, which we can drop subsequnetly
df_new.groupby(['restaurant']).size().sort_values(ascending=False)

restaurant
Camino                    2
Egg & Flour Pasta Bar     2
2Mesa Mexican Eatery      1
Rice N Roll Bistro        1
Richard’s Cafe            1
                         ..
Fuel Cafe 5th             1
GYRO MKE                  1
Goodkind                  1
Guadalajara Restaurant    1
Zocalo                    1
Length: 238, dtype: int64

In [76]:
df_new.drop_duplicates(keep='first', inplace=True)

In [78]:
df_new.to_csv('links.csv',index=False,header=True,encoding='utf8')

### Next, we scrape all the details about the restaurants with a web driver and the selenium package

In [85]:
df = pd.read_csv('links.csv')

In [86]:
df.tail()

,Restaurant,Complete_links
205,La Fondita De Milwaukee,https://www.yelp.com/biz/la-fondita-de-milwauk...
206,Taqueria Buenavista,https://www.yelp.com/biz/taqueria-buenavista-m...
207,Michael’s Family Restaurant,https://www.yelp.com/biz/michaels-family-resta...
208,Triskele’s,https://www.yelp.com/biz/triskeles-milwaukee?o...
209,El Beso Milwaukee - Mexican Restaurante & Cantina,https://www.yelp.com/biz/el-beso-milwaukee-mex...


In [140]:
'''options to avoide that actual windows are opened every time. create headless driver options'''
options = webdriver.FirefoxOptions()

#these options are set so that a browser doesn't actually open everytime we scrape
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

In [5]:
Links = df.Complete_links.to_list() #in the formula below Links is equal to Li
Restaurants = df.Restaurant.to_list() #in the formula below Restaurants is equal to Rest

In [141]:
''' Here we can make a little test run'''

time.sleep(3)
driver = webdriver.Firefox() #options=options 
driver.get('https://www.yelp.com/biz/highlands-cafe-wauwatosa?osq=Restaurants')

driver.maximize_window()
time.sleep(3)

#we first need to get rid of the privacy concern button so we click it
driver.find_element_by_css_selector('.onetrust-lg').click()

#then we wait until that click is processed, before we click the button where all the amenties are extended
time.sleep(3)
try:
    driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(6) > div:nth-child(2) > button:nth-child(2)').click()
except:
     print('no hidden amenities')

try:
    driver.find_element_by_css_selector(' div.padding-t3__373c0__1gw9E:nth-child(3) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1)').click()
    time.sleep(3)
    driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(5) > div:nth-child(2) > button:nth-child(2)').click()
    time.sleep(2)
    print('child 5')
    
except:
    print('no ammenities')
    

source = driver.page_source
soup1 = BeautifulSoup(source, 'html.parser')

no ammenities


In [24]:
driver.quit()

In [25]:
Li = df_new.Complete_links.to_list()
Rest = df_new.restaurant.to_list()

In [30]:
'''Selenium search automation can either fetch the search item from the list or text can be entered'''

df_rest = pd.DataFrame([],columns=['restaurant', 'link', 'claimed', 'number_of_reviews', 'number_of_photos', 'price', 'rating', 'categories', 'street', 'address', 'website', 'update', 'amenities'])


#This is so we can enumerate over the restaurant names
enum_extract = enumerate(Rest)

def getname():
    key = next(enum_extract)
    return str(key)

enum_rest = enumerate(Li)

def getlink():
    key = next(enum_rest)
    return str(key)
    


for i in Li:
    restaurant_link = []
    restaurant_update = []
    restaurant_description = []
    restaurant_numberofreviews = []        
    restaurant_photos = []
    restaurant_rating = []
    restaurant_street = []
    restaurant_adress = []

    restaurant_link = []
    restaurant_price = []
    restaurant_categories = []
    restaurant_name = [] 

    restaurant_claimed = [] 
    restaurant_amenities = []

    #so here we append the restaurant name
    restname = getname()
    restaurant_name.append(restname) 
    
    restlink = getlink()
    restaurant_link.append(restlink)
    
    try:
            #Here we initialize a headless driver
        time.sleep(3)
        driver = webdriver.Firefox(options=options) #options=options 
        driver.get(i)

        driver.maximize_window()
        time.sleep(3)

        #we first need to get rid of the privacy concern button so we click it
        driver.find_element_by_css_selector('.onetrust-lg').click()

        #then we wait until that click is processed, before we click the button where all the amenties are extended
        time.sleep(3)
        
        try:
            driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(6) > div:nth-child(2) > button:nth-child(2)').click()
            time.sleep(3)
        except:
            print('no hidden amenities')
        
        try:
            driver.find_element_by_css_selector(' div.padding-t3__373c0__1gw9E:nth-child(3) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1)').click()
            time.sleep(3)
            driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(5) > div:nth-child(2) > button:nth-child(2)').click()
            time.sleep(2)
            print('child 5')
        
        except:
            print('no ammenities')


        source = driver.page_source
        soup1 = BeautifulSoup(source, 'html.parser')


        #now we loop over the elements in the class
        subsoup = soup1.find_all(class_ = 'arrange-unit__373c0__1piwO border-color--default__373c0__2oFDT')
        amenities = []
        for k in range(len(subsoup)):
            subamenities=subsoup[k].get_text()
            amenities.append(subamenities)

        restaurant_amenities.append(amenities) 


        #number of reviews
        numberofreviews = soup1.select(".css-bq71j2") 
        restaurant_numberofreviews.append(numberofreviews[0].string)

        #number of photos
        photos = soup1.select(".css-ardur")
        restaurant_photos.append(photos[4].string) 

        #first and second address line
        location = soup1.select(".raw__373c0__3rcx7")
        restaurant_street.append(location[0].string) 
        restaurant_adress.append(location[1].string)

            #rating
        for item in soup1.select('[class*=container]'):
            if item.find('h4'):
                name = item.find('h4').get_text()
                ratings=soup1.select('[aria-label*=rating]')[0]
                rating=str(ratings)
                rating = re.search('aria-label="(.+?)star rating"', rating)
                if rating:
                    rating = rating.group(1)

        restaurant_rating.append(rating)

        #price
        price = soup1.select(".css-1xxismk")
        price=str(price)
        price = re.search('"css-1xxismk">(.+?)<', price)
        if price:
            price = price.group(1)
            restaurant_price.append(price) 

        #categories
        #this list needs to be inside the loop
        categories = []

        for item in soup1.select('[class*=container]'):
            if item.find('h1'):
                items=item.select('.css-166la90')
                for i in items:
                    categories.append(i.string)

                restaurant_categories.append(categories)

        if 'Unclaimed' not in categories:

            #updates
            update=soup1.select('.css-gdi06s')
            restaurant_update.append(update[0].string)             


            #homepage link
            link = soup1.select(".css-ac8spe")
            restaurant_link.append(link[6].string)


            restaurant_claimed.append('Claimed')

        else: 
            restaurant_update.append('None')
            restaurant_link.append('None')
            restaurant_claimed.append('Unclaimed')

            driver.quit()


    except: 
        restaurant_claimed.append('None')
        restaurant_update.append('None')
        restaurant_numberofreviews.append('None')
        restaurant_photos.append('None')
        restaurant_rating.append('None')
        restaurant_street.append('None')
        restaurant_adress.append('None')
        restaurant_link.append('None')
        restaurant_price.append('None')
        restaurant_categories.append('None')
        restaurant_amenities.append('None')
            
    finally:
        if len(restaurant_claimed) == 0:
            restaurant_claimed.append('None')
                
        if len(restaurant_update) == 0:
            restaurant_update.append('None')
                
        if len(restaurant_numberofreviews) == 0:
            restaurant_numberofreviews.append('None')
                
        if len(restaurant_photos) == 0:
            restaurant_photos.append('None')
                
        if len(restaurant_rating) == 0:
            restaurant_rating.append('None')
                
        if len(restaurant_street) == 0:
            restaurant_street.append('None')
                
        if len(restaurant_adress) == 0:
            restaurant_adress.append('None')
                
        if len(restaurant_link) == 0:
            restaurant_link.append('None')
                
        if len(restaurant_price) == 0:
            restaurant_price.append('None')
                
        if len(restaurant_categories) == 0:
            restaurant_categories.append('None')
                
        if len(restaurant_amenities) == 0:
            restaurant_amenities.append('None')
                
        entry_list = list(zip(restaurant_name, restaurant_link, restaurant_claimed, restaurant_numberofreviews, restaurant_photos, restaurant_price, restaurant_rating, restaurant_categories, restaurant_street, restaurant_adress, restaurant_link,restaurant_update, restaurant_amenities))

            
        df_entry = pd.DataFrame(entry_list,columns=['restaurant', 'link', 'claimed', 'number_of_reviews', 'number_of_photos', 'price', 'rating', 'categories', 'street', 'address', 'website', 'update', 'amenities'])
            
        frames = [df_rest, df_entry]
        df_rest = pd.concat(frames)
        time.sleep(5)

no hidden amenities
no ammenities
no ammenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no ammenities
no ammenities
no ammenities
child 5
no ammenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
child 5
no ammenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no hidden amenities
no ammenities


In [103]:
#with this code we can always check whether we got blocked by yelp, the status code should be equal to 200
res  = requests.get('https://www.yelp.com/biz/lazos-taco-shack-milwaukee?osq=Restaurants')

res.status_code

200

### We scraped the restaurants in several gos, which is why we now have to concatinate the different frames and clean them up

In [32]:
df_rest.tail()

,restaurant,link,claimed,number_of_reviews,number_of_photos,price,rating,categories,street,address,website,update,amenities
0,"(30, 'Skyway Cafe')","(30, 'https://www.yelp.com/biz/skyway-cafe-mil...",Unclaimed,13 reviews,See 23 photos,$,4.5,"[Unclaimed, Cafes]",626 East Wisconsin Ave,Ground floor,"(30, 'https://www.yelp.com/biz/skyway-cafe-mil...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(31, 'The Green Kitchen')","(31, 'https://www.yelp.com/biz/the-green-kitch...",Unclaimed,164 reviews,See 77 photos,$,4,"[Unclaimed, Vegetarian, Sandwiches]",400 N Water St,"Milwaukee, WI 53202","(31, 'https://www.yelp.com/biz/the-green-kitch...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(32, 'Thainamite - Wauwatosa')","(32, 'https://www.yelp.com/biz/thainamite-wauw...",Claimed,141 reviews,See 134 photos,$$,3.5,"[Thai, Sushi Bars, Cocktail Bars]",8725 W North Ave,"Wauwatosa, WI 53226","(32, 'https://www.yelp.com/biz/thainamite-wauw...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(33, 'Deli 1614')","(33, 'https://www.yelp.com/biz/deli-1614-milwa...",Unclaimed,17 reviews,See 12 photos,$,4.5,"[Unclaimed, Delis]",1614 W Walnut St,"Milwaukee, WI 53205","(33, 'https://www.yelp.com/biz/deli-1614-milwa...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(34, 'Riverfront Pizzeria Bar & Grill')","(34, 'https://www.yelp.com/biz/riverfront-pizz...",None,136 reviews,See 110 photos,$$,3.5,"[Pizza, Bars]",509 E Erie St,"Milwaukee, WI 53202","(34, 'https://www.yelp.com/biz/riverfront-pizz...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."


In [34]:
df10 = df_rest

In [35]:
df10.to_csv('usable10.csv')

In [36]:
df1 = pd.read_csv('usable.csv')
df2 = pd.read_csv('usable2.csv')
df3 = pd.read_csv('usable3.csv')
df4 = pd.read_csv('usable4.csv')
df5 = pd.read_csv('usable5.csv')
df6 = pd.read_csv('usable6.csv')
df7 = pd.read_csv('usable7.csv')
df8 = pd.read_csv('usable8.csv')
df9 = pd.read_csv('usable9.csv')
df10 = pd.read_csv('usable10.csv')

In [48]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]
dfdf = pd.concat(frames)

In [49]:
len(dfdf)

297

In [50]:
dfdf.head()
dfdf = dfdf.loc[:, ~dfdf.columns.str.contains('^Unnamed')]
dfdf['restaurant'] = dfdf['restaurant'].astype('str') 

In [53]:
import regex as re

new_restaurants = dfdf.restaurant.to_list()

'''stripping anything werid from the search term'''
def searchconcat(x):
    x = x.replace(',', '')\
    .replace("'","")\
    .replace('(','')\
    .replace(')','')\
    .replace('0', '')\
    .replace('1', '')\
    .replace('2', '')\
    .replace('3', '')\
    .replace('4', '')\
    .replace('5', '')\
    .replace('6', '')\
    .replace('7', '')\
    .replace('8', '')\
    .replace('9', '')\
    .strip()
    
    return str(x)



new_ii = []
for i in new_restaurants:
    new_i=searchconcat(i)
    new_ii.append(new_i)
    

dfdf['restaurant'] = new_ii

In [56]:
dfdf.drop_duplicates(subset=['restaurant'], keep='last', inplace = True)

In [62]:
len(dfdf)

253

### Then here is the final dataframe

In [67]:
dfdf.tail(50)

,restaurant,link,claimed,number_of_reviews,number_of_photos,price,rating,categories,street,address,website,update,amenities
6,Tiki Bar MKE Public Market,"(6, 'https://www.yelp.com/biz/tiki-bar-mke-pub...",Unclaimed,5 reviews,See 29 photos,None,5.0,"['Unclaimed', 'Bars', 'Seafood']",401 N Broadway,"Milwaukee, WI 53202","(6, 'https://www.yelp.com/biz/tiki-bar-mke-pub...",None,"['', 'For BusinessesWrite a ReviewLog InSign U..."
7,Alem Ethiopian Village,"(7, 'https://www.yelp.com/biz/alem-ethiopian-v...",Claimed,279 reviews,See 147 photos,$$,4.5,"['Ethiopian', 'Vegetarian']",307 E Wisconsin Ave,"Milwaukee, WI 53202","(7, 'https://www.yelp.com/biz/alem-ethiopian-v...",NaN,"['', 'For BusinessesWrite a ReviewLog InSign U..."
8,East Town Kitchen & Bar,"(8, 'https://www.yelp.com/biz/east-town-kitche...",Claimed,75 reviews,See 120 photos,$$,4.0,"['American (New)', 'Beer Bar', 'Cocktail Bars']",323 E Wisconsin Ave,"Milwaukee, WI 53202","(8, 'https://www.yelp.com/biz/east-town-kitche...",Milwaukee Marriott Downtown,"['', 'For BusinessesWrite a ReviewLog InSign U..."
9,Festival Foods,"(9, 'https://www.yelp.com/biz/festival-foods-m...",Claimed,4 reviews,See 43 photos,None,3.5,"['Grocery', 'Bakeries', 'Delis']",11111 W Greenfield Ave,"Milwaukee, WI 53214","(9, 'https://www.yelp.com/biz/festival-foods-m...",NaN,"['', 'For BusinessesWrite a ReviewLog InSign U..."
11,Egg & Flour Pasta Bar,"(11, 'https://www.yelp.com/biz/egg-and-flour-p...",Unclaimed,38 reviews,See 98 photos,None,4.5,"['Unclaimed', 'Italian']",2273 S Howell Ave,"Milwaukee, WI 53207","(11, 'https://www.yelp.com/biz/egg-and-flour-p...",None,"['', 'For BusinessesWrite a ReviewLog InSign U..."
0,Black Sheep,"(0, 'https://www.yelp.com/biz/black-sheep-milw...",Claimed,256 reviews,See 360 photos,$$,4,"['Wine Bars', 'American (New)', 'Breakfast & B...",216 S 2nd St,"Milwaukee, WI 53204","(0, 'https://www.yelp.com/biz/black-sheep-milw...","""Open for dine-in, carryout, curbside and deli...","['', 'For BusinessesWrite a ReviewLog InSign U..."
1,Taste Amir’s Roti,"(1, 'https://www.yelp.com/biz/taste-amirs-roti...",Unclaimed,2 reviews,See 4 photos,None,5,"['Unclaimed', 'Food Trucks', 'Indian']",3953 S 20th St,"Milwaukee, WI 53221","(1, 'https://www.yelp.com/biz/taste-amirs-roti...",None,"['', 'For BusinessesWrite a ReviewLog InSign U..."
2,Wild Roots,"(2, 'https://www.yelp.com/biz/wild-roots-west-...",Claimed,72 reviews,See 134 photos,$$,4.5,"['Breakfast & Brunch', 'American (New)', 'Beer...",6807 W Becher St,"West Allis, WI 53219","(2, 'https://www.yelp.com/biz/wild-roots-west-...",NaN,"['', 'For BusinessesWrite a ReviewLog InSign U..."
3,The Social,"(3, 'https://www.yelp.com/biz/the-social-milwa...",Unclaimed,1 review,See 6 photos,None,5,"['Unclaimed', 'American (Traditional)', 'Beer ...",611 N Broadway,"Milwaukee, WI 53202","(3, 'https://www.yelp.com/biz/the-social-milwa...",None,"['', 'For BusinessesWrite a ReviewLog InSign U..."
4,Easy Tyger,"(4, 'https://www.yelp.com/biz/easy-tyger-milwa...",Claimed,144 reviews,See 226 photos,$$,4.5,"['Ramen', 'Asian Fusion', 'Gastropubs']",1230 E Brady St,"Milwaukee, WI 53202","(4, 'https://www.yelp.com/biz/easy-tyger-milwa...",NaN,"['', 'For BusinessesWrite a ReviewLog InSign U..."


In [63]:
dfdf.to_csv('milwaukee.restaurants.yelp.csv')

In [66]:
dflist = dfdf.restaurant.to_list()
len(set(dflist))

253

### Now on to the individual reviews

In [74]:
df = pd.read_csv('mainlinks.csv')

In [75]:
'''then we complete the links colum for the full URL'''

df['Complete_links']= df['link'].apply(lambda x:"https://www.yelp.com"+str(x))

In [76]:
Links = df.Complete_links.to_list() #in the formula below Links is equal to Li
Restaurants = df.restaurant.to_list() #in the formula below Restaurants is equal to Rest

In [111]:
restrest = Restaurants[:]
linklink = Links[:]

In [78]:
url_yelp = 'https://www.yelp.com/biz/sweetdiner-milwaukee?osq=Restaurants'
html = requests.get(url_yelp)
soup2 = BeautifulSoup(html.content, 'lxml')

html.status_code

200

In [112]:
import pyautogui

In [113]:
''' Now we scrape the reviews, this time we do not have to make use of the selenoim package, we onyl work with URLs which should make this process a bit easier'''

#we initialize a dataframe, but of course outside the loop
df1 = pd.DataFrame([],columns=['Restaurant', 'Review'])

enum_extract = enumerate(restrest)


failedurls = []

def getname():
    key = next(enum_extract)
    return str(key)


for l in linklink:
    url_yelp = l
    html = requests.get(url_yelp)
    
    #soup2 is the base website
    soup2 = BeautifulSoup(html.content, 'lxml')
    
    if html.status_code == 200:
        print('status passed')
        length = ''
        length1 = 0
        length2 = 0
        reviews = []
        try: 
            ## we search for the number of sites to loop through on the base website soup2
            length=soup2.select('.css-e81eai')
            length = str(length)
            #we use a regular expression to filter out the total length of the review
            length = re.search('>(\d+) of (\d+)<', length)
            if length:
                length1 = length.group(1)
                length2 = int(length.group(2))
                
            if length2 > 20:
                length2 = 20
            
            print(length2, length1)

            
            for i in range(0,length2*10,10):
                html = requests.get(url_yelp+'&start='+str(i))
                soup3 = BeautifulSoup(html.content, 'lxml')
                soupstring = str(soup3)
                #this is the pattern for all the reviews, it will find all matches so we don't have to worry about not grabbing one. Though we will have to clean this up later
                subsoup = re.findall('"markupDisplayName":(.+?),"voterText":', soupstring)
                reviews += subsoup
                time.sleep(5)

            
            #outside of the last loop, we create a df, where we zip the lists, and then we append them to the df
            
            restname = getname()
            keys = [restname]*int(len(reviews))
            review_list = list(zip(keys, reviews))

            df2=pd.DataFrame(review_list, columns=['Restaurant','Review'])

            frames= [df1, df2]
            df1 = pd.concat(frames)
            print(restname)
            time.sleep(3)  
                   
        except:
            length = ''
            length1 = 0
            length2 = 0
            reviews = []
            restname = getname()
            print(f'No reviews for {restname}')
            time.sleep(5)
              
        #In case we have lots of reviews, we just scrape the first 200
         
    else:
        length = ''
        length1 = 0
        length2 = 0
        reviews = []
        print(f'We got blocked for {url_yelp}')
        failedurls.append(url_yelp)
        
        pyautogui.moveTo(x=1145, y=14) 
        pyautogui.click()
        time.sleep(2)
        pyautogui.moveTo(x=1158, y=105) 
        pyautogui.moveTo(x=1350, y=105, duration = 3)
        time.sleep(2)
        pyautogui.moveTo(x=1350, y=220, duration=2)
        pyautogui.click()
        time.sleep(2)
        pyautogui.alert(text='Enter password', title='Enter password', button='OK')

        print('continue')
        #pyautogui.write('Ilemlbseicmzt@2501527uva', interval=0.25)
        time.sleep(1)
        #pyautogui.press('enter')
        time.sleep(6)

status passed
20 1
No reviews for (0, 'Swingin’ Door Exchange')
status passed
20 1
No reviews for (1, 'Story Hill BKC')


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-113-169a291bf1a1>", line 52, in <module>
    time.sleep(5)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/julia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-113-169a291bf1a1>", line 75, in <module>
    time.sleep(5)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/julia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/julia/opt/anacon

TypeError: object of type 'NoneType' has no len()

In [109]:
len(df1)

46

,Restaurant,Review
0,"(0, 'Lazo’s Taco Shack')","""Becky Q."",""displayLocation"":""Milwaukee, WI"",""..."
1,"(0, 'Lazo’s Taco Shack')","""Jason S."",""displayLocation"":""Milwaukee, WI"",""..."
2,"(0, 'Lazo’s Taco Shack')","""Kelli R."",""displayLocation"":""Milwaukee, WI"",""..."
3,"(0, 'Lazo’s Taco Shack')","""Jeff M."",""displayLocation"":""Milwaukee, WI"",""a..."
4,"(0, 'Lazo’s Taco Shack')","""Raelee G."",""displayLocation"":""Scottsdale, AZ""..."
5,"(0, 'Lazo’s Taco Shack')","""Bradley A."",""displayLocation"":""Baton Rouge, L..."
6,"(0, 'Lazo’s Taco Shack')","""Fred A."",""displayLocation"":""Tallahassee, FL"",..."
7,"(0, 'Lazo’s Taco Shack')","""Rick R."",""displayLocation"":""Milwaukee, WI"",""a..."
8,"(0, 'Lazo’s Taco Shack')","""ryan M."",""displayLocation"":""Union, NC"",""altTe..."
9,"(0, 'Lazo’s Taco Shack')","""Mandy M."",""displayLocation"":""Middleton, WI"",""..."


In [69]:
df_7 = df1

In [70]:
len(df_7)

181

In [71]:
df_7.to_csv('df_7.reviews.csv')

In [2]:
import pandas as pd

In [38]:
df_1= pd.read_csv('df_1.reviews.csv')
df_2= pd.read_csv('df_2.reviews.csv')
df_3= pd.read_csv('df_3.reviews.csv')
df_4= pd.read_csv('df_4.reviews.csv')
df_5 = pd.read_csv('df_5.reviews.csv')
df_6 = pd.read_csv('df_6.reviews.csv')
df_7 = pd.read_csv('df_7.reviews.csv')

In [39]:
df1 = pd.concat([df_1,df_2, df_3, df_4, df_5, df_6, df_7])
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

In [40]:
df1.head()

,Restaurant,Review
0,"(0, 'Swingin’ Door Exchange')","""KC and Shelly S."",""photoSrc"":""https://s3-medi..."
1,"(0, 'Swingin’ Door Exchange')","""Hart P."",""displayLocation"":""Gallatin, TN"",""al..."
2,"(0, 'Swingin’ Door Exchange')","""Warren D."",""displayLocation"":""Milwaukee, WI"",..."
3,"(0, 'Swingin’ Door Exchange')","""Daniela B."",""displayLocation"":""Brookfield, WI..."
4,"(0, 'Swingin’ Door Exchange')","""Ashley B."",""displayLocation"":""Chicago, IL"",""a..."


In [41]:
import regex as re

new_restaurants = df1.Restaurant.to_list()

'''stripping anything werid from the search term'''
def searchconcat(x):
    x = x.replace(',', '')\
    .replace("'","")\
    .replace('(','')\
    .replace(')','')\
    .replace('0', '')\
    .replace('1', '')\
    .replace('2', '')\
    .replace('3', '')\
    .replace('4', '')\
    .replace('5', '')\
    .replace('6', '')\
    .replace('7', '')\
    .replace('8', '')\
    .replace('9', '')\
    .strip()
    
    return str(x)



new_ii = []
for i in new_restaurants:
    new_i=searchconcat(i)
    new_ii.append(new_i)
    

df1['Restaurant'] = new_ii

In [42]:
df1.head()

,Restaurant,Review
0,Swingin’ Door Exchange,"""KC and Shelly S."",""photoSrc"":""https://s3-medi..."
1,Swingin’ Door Exchange,"""Hart P."",""displayLocation"":""Gallatin, TN"",""al..."
2,Swingin’ Door Exchange,"""Warren D."",""displayLocation"":""Milwaukee, WI"",..."
3,Swingin’ Door Exchange,"""Daniela B."",""displayLocation"":""Brookfield, WI..."
4,Swingin’ Door Exchange,"""Ashley B."",""displayLocation"":""Chicago, IL"",""a..."


In [43]:
''' Lets cut out the photos already, cause that would be too much to parse'''

df1[df1['Restaurant'] == 'Swingin’ Door Exchange']['Review'].values[8]

'"Laura L.","displayLocation":"Larkspur, CA","altText":"Laura L.","userUrl":"/user_details?userid=9M_Y3HlDnWmIYHQhBFqSkA","partnerAlias":null,"friendCount":67,"photoCount":13,"reviewCount":44,"eliteYear":null},"comment":{"text":"Great food- as always. My spouse reeeeAaaaaallllllyyyy hopes they bring bacj the Hungarian sausage. Food is amazing. Get the carrots. Get the chili. Get the tato pancakes. I love the chicken siracha (spelled correctly on their menu).<br&gt;Kick a$$ Tom Collins","language":"en"},"localizedDate":"3/27/2021","localizedDateVisited":null,"rating":5,"photos":[{"src":"https://s3-media0.fl.yelpcdn.com/bphoto/ofWI7OsdXXdfdM31BsiJmQ/300s.jpg","link":"/biz_photos/swingin-door-exchange-milwaukee?reviewid=U-yMLCJ9XWwA995AGXmfqw&amp;select=ofWI7OsdXXdfdM31BsiJmQ","altText":"Photo of Swingin\' Door Exchange - Milwaukee, WI, United States","width":300,"height":300,"caption":null}],"lightboxMediaItems":[{"business":{"alias":"swingin-door-exchange-milwaukee","id":"mgVA9tNReJqiUG

In [44]:
df1['username'] = df1['Review'].apply(lambda x: re.findall('"(.+?)"', x)[0])

In [45]:
df1['text'] = df1['Review'].apply(lambda x: re.findall('"comment":{"text":(.+?),"language":"', x)[0])

In [46]:
df1['rating'] = df1['Review'].apply(lambda x: re.findall('"rating":(.+?),', x)[0]) #"photos"

In [47]:
df1['userfeedback'] = df1['Review'].apply(lambda x: re.findall('"feedback":{"counts":{(.+?)},', x)[0])

df1['userfeedback_useful'] = df1['userfeedback'].apply(lambda x: re.findall('"useful":(.+?),', x)[0])
df1['userfeedback_funny'] = df1['userfeedback'].apply(lambda x: re.findall('"funny":(.+?),', x)[0])
df1['userfeedback_cool'] = df1['userfeedback'].apply(lambda x: re.findall('"cool":(.+?)', x)[0])

In [48]:
df1['reviewtotalPhotos'] = df1['Review'].apply(lambda x: re.findall('"totalPhotos":(.+?),', x)[0])

In [49]:
df1['peerfeedback'] = df1['Review'].apply(lambda x: re.findall('"userFeedback":{(.+?)}', x)[0])


df1['peerfeedback_useful'] = df1['peerfeedback'].apply(lambda x: re.findall('"useful":(.+?),', x)[0])
df1['peerfeedback_funny'] = df1['peerfeedback'].apply(lambda x: re.findall('"funny":(.+?),', x)[0])
df1['peerfeedback_cool'] = df1['peerfeedback'].apply(lambda x: re.findall('"cool":(.+?)', x)[0])

In [50]:
df1['language'] = df1['Review'].apply(lambda x: re.findall('"language":"(.+?)"}', x)[0])

In [51]:
df1['Date'] = df1['Review'].apply(lambda x: re.findall('"localizedDate":"(.+?)","', x)[0])

In [52]:
df1['userlocation'] = df1['Review'].apply(lambda x: re.findall('"displayLocation":"(.+?)"', x)[0])

In [53]:
df1['friendCount'] = df1['Review'].apply(lambda x: re.findall('"friendCount":(.+?),', x)[0])

In [54]:
df1['photoCount'] = df1['Review'].apply(lambda x: re.findall('"photoCount":(.+?),', x)[0])

In [55]:
df1['reviewCount'] = df1['Review'].apply(lambda x: re.findall('"reviewCount":(.+?),', x)[0])

In [56]:
df1['eliteYear'] = df1['Review'].apply(lambda x: re.findall('"eliteYear":(.+?)},', x)[0])

In [57]:
def elite(x):
    x=re.findall('"isCurrentlyElite":(.+?),', x)
    try:
        if x[0]:
            return x[0]
    except:
        x ='false'
        return x

In [58]:
df1['isCurrentlyElite'] = df1['Review'].apply(lambda x: elite(x))

In [59]:
df1.head(10)

,Restaurant,Review,username,text,rating,userfeedback,userfeedback_useful,userfeedback_funny,userfeedback_cool,reviewtotalPhotos,...,peerfeedback_funny,peerfeedback_cool,language,Date,userlocation,friendCount,photoCount,reviewCount,eliteYear,isCurrentlyElite
0,Swingin’ Door Exchange,"""KC and Shelly S."",""photoSrc"":""https://s3-medi...",KC and Shelly S.,"""I&amp;#39;ve known about this place for quite...",4.5,"""useful"":2,""funny"":1,""cool"":2",2,1,2,0,...,false,f,en,3/25/2021,"Milwaukee, WI",155,146,26,2021,false
1,Swingin’ Door Exchange,"""Hart P."",""displayLocation"":""Gallatin, TN"",""al...",Hart P.,"""Great experience. Indoor dining at the bar - ...",5,"""useful"":1,""funny"":0,""cool"":1",1,0,1,3,...,false,f,en,8/11/2020,"Gallatin, TN",57,1339,718,2021,true
2,Swingin’ Door Exchange,"""Warren D."",""displayLocation"":""Milwaukee, WI"",...",Warren D.,"""I selected Swingin&amp;#39; Door Exchange for...",5,"""useful"":8,""funny"":2,""cool"":7",8,2,7,1,...,false,f,en,5/12/2020,"Milwaukee, WI",121,361,527,2021,true
3,Swingin’ Door Exchange,"""Daniela B."",""displayLocation"":""Brookfield, WI...",Daniela B.,"""I ordered the fish fry on a Friday for me and...",5,"""useful"":0,""funny"":0,""cool"":1",0,0,1,1,...,false,f,en,5/30/2021,"Brookfield, WI",17,472,235,2021,true
4,Swingin’ Door Exchange,"""Ashley B."",""displayLocation"":""Chicago, IL"",""a...",Ashley B.,"""This four star review is simply for their foo...",4,"""useful"":0,""funny"":0,""cool"":0",0,0,0,3,...,false,f,en,6/1/2021,"Chicago, IL",355,377,154,2021,true
5,Swingin’ Door Exchange,"""Meagan M."",""displayLocation"":""Lower East Side...",Meagan M.,"""Went here after a day of drinking in Milwauke...",4,"""useful"":0,""funny"":0,""cool"":1",0,0,1,1,...,false,f,en,5/15/2021,"Lower East Side, Milwaukee, WI",5,105,58,2021,true
6,Swingin’ Door Exchange,"""Hannah W."",""displayLocation"":""Brookfield, WI""...",Hannah W.,"""My parents &amp;amp; I came to Swingin&amp;#3...",4,"""useful"":0,""funny"":0,""cool"":0",0,0,0,2,...,false,f,en,4/24/2021,"Brookfield, WI",12,54,70,2021,true
7,Swingin’ Door Exchange,"""Matthew C."",""displayLocation"":""Kalamazoo, MI""...",Matthew C.,"""This is a pretty cool bar/restaurant! It&amp;...",3,"""useful"":1,""funny"":1,""cool"":0",1,1,0,2,...,false,f,en,3/24/2021,"Kalamazoo, MI",116,961,260,2021,true
8,Swingin’ Door Exchange,"""Laura L."",""displayLocation"":""Larkspur, CA"",""a...",Laura L.,"""Great food- as always. My spouse reeeeAaaaaal...",5,"""useful"":1,""funny"":0,""cool"":1",1,0,1,1,...,false,f,en,3/27/2021,"Larkspur, CA",67,13,44,null,false
9,Swingin’ Door Exchange,"""James D."",""displayLocation"":""Denver, CO"",""alt...",James D.,"""As a frequent traveler; one looks to find pla...",5,"""useful"":0,""funny"":0,""cool"":1",0,0,1,0,...,false,f,en,6/1/2021,"Denver, CO",120,11,43,null,false


In [60]:
len(df1)

12392

In [26]:
dfx =df1[(df1['rating'] == '1') | (df1['rating'] == '2')] 
dfy = df1[(df1['rating'] == '3') | (df1['rating'] == '4')] 
dfz = df1[(df1['rating'] == '5') | (df1['rating'] == '6')]

frames = [dfx, dfy, dfz]

In [27]:
df1 = pd.concat(frames)

In [61]:
len(df1)

12392

In [29]:
df1['rating'] = df1['rating'].astype('int') 

In [30]:
df_unclened = df1[['Restaurant', 'username', 'Review']]

In [31]:
df_unclened.to_csv('df_uncleaned_reviews.csv')

In [62]:
df1.pop('Review')
df1.pop('userfeedback')
df1.pop('peerfeedback')

0      "useful":false,"funny":false,"cool":false
1      "useful":false,"funny":false,"cool":false
2      "useful":false,"funny":false,"cool":false
3      "useful":false,"funny":false,"cool":false
4      "useful":false,"funny":false,"cool":false
                         ...                    
176    "useful":false,"funny":false,"cool":false
177    "useful":false,"funny":false,"cool":false
178    "useful":false,"funny":false,"cool":false
179    "useful":false,"funny":false,"cool":false
180    "useful":false,"funny":false,"cool":false
Name: peerfeedback, Length: 12392, dtype: object

In [64]:
df1.Restaurant.nunique()

223

In [65]:
pwd()

'/Users/julia/Downloads'

In [66]:
df1.to_excel(r'/Users/julia/Downloads/all_reviews.xlsx', index = False)

In [67]:
df1.to_csv('all_reviews.csv')

In [112]:
df = pd.read_csv('mainlinks.csv')

In [37]:
df1.head(30)

,Restaurant,username,text,rating,userfeedback_useful,userfeedback_funny,userfeedback_cool,reviewtotalPhotos,peerfeedback_useful,peerfeedback_funny,peerfeedback_cool,language,Date,userlocation,friendCount,photoCount,reviewCount,eliteYear,isCurrentlyElite
25,Swingin’ Door Exchange,Denim M.,"""It was the driest burger in my life) I was ve...",2,0,0,0,1,false,false,f,en,3/13/2021,"Kearney, NE",7,9,10,null,false
205,Swingin’ Door Exchange,David N.,"""We were downtown for a show at the Riverside....",2,0,0,0,0,false,false,f,en,11/17/2018,"Muskego, WI",0,null,2,null,false
293,Story Hill BKC,Noah K.,"""The manager was one of the rudest people I&am...",1,0,0,0,0,false,false,f,en,1/7/2021,"Story Hill, Milwaukee, WI",4,null,2,null,false
382,Story Hill BKC,Louisa E.,"""I had read the reviews and expected an amazin...",2,0,0,0,0,false,false,f,en,2/26/2018,"Milwaukee, WI",29,16,159,2021,false
445,Mad Rooster Cafe,Jen W.,"""I live 5 minutes away from this place, and ju...",2,1,0,0,0,false,false,f,en,1/25/2021,"Milwaukee, WI",2,4,13,null,false
483,Mad Rooster Cafe,Kay F.,"""I always give them a five...this is a COVID d...",2,0,0,0,0,false,false,f,en,10/15/2020,"Milwaukee, WI",0,5,15,null,false
489,Mad Rooster Cafe,Amanda W.,"""This place was ok, I would not recommend the ...",2,0,0,0,1,false,false,f,en,9/8/2020,"West Milwaukee, WI",28,1,1,null,false
505,Mad Rooster Cafe,Karl H.,"""The first couple times I got takeout from her...",1,1,1,0,1,false,false,f,en,9/5/2019,"Milwaukee, WI",23,9,127,null,false
546,Mad Rooster Cafe,Jessica G.,"""Ugh.<br&gt;<br&gt;I came here for a family br...",1,3,1,0,0,false,false,f,en,10/10/2019,"Bay View, Milwaukee, WI",8,1,49,null,false
562,Mad Rooster Cafe,Kiera B.,"""During Covid times I would expect waitresses ...",1,0,0,0,0,false,false,f,en,8/8/2020,"Glendale, WI",94,null,5,null,false


In [113]:
df.restaurant.nunique()

238

In [146]:
''' Now we need to comapre the difference'''

old_rest = df.restaurant.to_list()

new_rest = df1.Restaurant.to_list()


new_rest = df_mainlinks.restaurant.to_list()

df_new = df1[~df1['Restaurant'].isin(old_rest)]

#df_new

No, List1 doesn't have all elements of the List2.


In [1]:
import pandas as pd
df1 = pd.read_csv('all_reviews.csv')

In [2]:
df1.head()

,Unnamed: 0,Restaurant,Review_1,Review_2
0,0,Swingin’ Door Exchange,"""KC and Shelly S."",""photoSrc"":""https://s3-medi...",NaN
1,1,Swingin’ Door Exchange,"""Hart P."",""displayLocation"":""Gallatin, TN"",""al...",""":""Hart P."",""eliteYear"":2021,""friendCount"":57,..."
2,2,Swingin’ Door Exchange,"""Warren D."",""displayLocation"":""Milwaukee, WI"",...",""":""Warren D."",""eliteYear"":2021,""friendCount"":1..."
3,3,Swingin’ Door Exchange,"""Daniela B."",""displayLocation"":""Brookfield, WI...",""":""Daniela B."",""eliteYear"":2021,""friendCount"":..."
4,4,Swingin’ Door Exchange,"""Ashley B."",""displayLocation"":""Chicago, IL"",""a...",""":""Ashley B."",""eliteYear"":2021,""friendCount"":3..."


In [3]:
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
df1['Restaurant'] = df1['Restaurant'].astype('str') 

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
df1[df1['Restaurant'] == 'Swingin’ Door Exchange']['Review_2'].values[4:5]

array(['":"Ashley B.","eliteYear":2021,"friendCount":355,"id":"GMCYglf7iCtFtnlZbwUNcw","isCurrentlyElite":true,"reviewCount":154,"url":"/user_details?userid=GMCYglf7iCtFtnlZbwUNcw"},"userVote":"NoVote"},{"business":{"alias":"swingin-door-exchange-milwaukee","id":"mgVA9tNReJqiUGx33nfqqQ","name":"Swingin\' Door Exchange"},"businessAvatar":{"altText":"Swingin\' Door Exchange","height":30,"src":"https://s3-media0.fl.yelpcdn.com/bphoto/nQglUY8Hik8W41OrJZ-s9Q/30s.jpg","srcSet":"https://s3-media0.fl.yelpcdn.com/bphoto/nQglUY8Hik8W41OrJZ-s9Q/ss.jpg 1.33x,https://s3-media0.fl.yelpcdn.com/bphoto/nQglUY8Hik8W41OrJZ-s9Q/60s.jpg 2.00x,https://s3-media0.fl.yelpcdn.com/bphoto/nQglUY8Hik8W41OrJZ-s9Q/90s.jpg 3.00x","url":"/biz/swingin-door-exchange-milwaukee","width":30},"caption":"Kts Buffalo Chicken Dip","helpfulCount":0,"id":"9e25z9b05mWIOFHfsm8uHw","timeUploaded":"June 1, 2021","type":"photo","url":"https://s3-media0.fl.yelpcdn.com/bphoto/9e25z9b05mWIOFHfsm8uHw/o.jpg","user":{"avatar":{"linkUrl":"/